In [ ]:
from ultralytics import YOLO

# Load the YOLO12 model

import torch
import torchvision

model = YOLO('/Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/last_model_train12052025.pt')
model.predict(source="/Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/אימון תמונות מהאופנוע/images/val",save_txt=True)




image 1/451 /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/אימון תמונות מהאופנוע/images/val/104_jpg.rf.b693b6ff25a60b429be7737be4b5fe20.jpg: 512x512 1 Manhole, 1 Pothole, 590.7ms
image 2/451 /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/אימון תמונות מהאופנוע/images/val/110_jpg.rf.91a5804ca1cbe32a0a0989d04dee39b6.jpg: 512x512 1 Crosswalk Blur, 845.4ms
image 3/451 /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/אימון תמונות מהאופנוע/images/val/123_jpg.rf.5cc47ce96c20322a082aa6b983bac39e.jpg: 512x512 (no detections), 563.4ms
image 4/451 /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/אימון תמונות מהאופנוע/images/val/125_jpg.rf.bdb844ddc41645e50e61e24d99b8436b.jpg: 512x512 1 Pothole, 599.3ms
image 5/451 /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/אימון תמונות מהאופנוע/images/val/12_jpg.rf.6c3c2f8f472803f27c058a951aa889e2.jpg: 512x512 

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Alligator Crack', 1: 'Block Crack', 2: 'Crosswalk Blur', 3: 'Lane Blur', 4: 'Longitudinal Crack', 5: 'Manhole', 6: 'Patch Repair', 7: 'Pothole', 8: 'Transverse Crack', 9: 'Wheel Mark Crack'}
 obb: None
 orig_img: array([[[162, 167, 165],
         [183, 186, 184],
         [192, 190, 190],
         ...,
         [255, 255, 195],
         [255, 255, 195],
         [255, 255, 195]],
 
        [[174, 177, 175],
         [167, 168, 166],
         [166, 162, 161],
         ...,
         [255, 255, 195],
         [255, 255, 195],
         [255, 255, 195]],
 
        [[167, 168, 164],
         [160, 159, 155],
         [159, 154, 151],
         ...,
         [255, 255, 195],
         [255, 255, 195],
         [255, 255, 195]],
 
        ...,
 
        [[ 42,  34,  27],
         [ 42,  34,  27],
         [ 43,  35,  28],
         ...,
         

: 

In [9]:
import os
import glob
import random
import shutil

# Parameters
base_dir = '/Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/אימון תמונות מהאופנוע'  # directory containing 'images' and 'labels' subdirs
output_dir = '/Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/dashcamera'
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1
random_seed = 42

# Paths
images_dir = os.path.join(base_dir, 'images')
labels_dir = os.path.join(base_dir, 'labels')

# Gather all image files
image_extensions = ('*.jpg', '*.jpeg', '*.png')
images = []
for ext in image_extensions:
    images.extend(glob.glob(os.path.join(images_dir, ext)))

# Shuffle for randomness
random.seed(random_seed)
random.shuffle(images)

# Compute split boundaries
total = len(images)
train_end = int(total * train_ratio)
val_end = train_end + int(total * val_ratio)

splits = {
    'train': images[:train_end],
    'val': images[train_end:val_end],
    'test': images[val_end:],
}

# Create and populate split directories
for split, img_paths in splits.items():
    img_out = os.path.join(output_dir, 'images', split)
    lbl_out = os.path.join(output_dir, 'labels', split)
    os.makedirs(img_out, exist_ok=True)
    os.makedirs(lbl_out, exist_ok=True)
    
    for img_path in img_paths:
        fname = os.path.basename(img_path)
        name, _ = os.path.splitext(fname)
        lbl_path = os.path.join(labels_dir, f'{name}.txt')
        
        # Copy image
        shutil.copy(img_path, os.path.join(img_out, fname))
        
        # Copy label if exists
        if os.path.exists(lbl_path):
            shutil.copy(lbl_path, os.path.join(lbl_out, f'{name}.txt'))



In [2]:
import os
import glob

# 👉 תעדכן לשביל אל תיקיית ה-patch בלבד שלך
PATCH_BASE = '/Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/patchs-2'  # מכיל train/, valid/, test/ עם images/ ו־labels/ בפנים

# המפה של split בתיקיות שלך
splits = {
    'train': ('train/images', 'train/labels'),
    'valid': ('valid/images', 'valid/labels'),
    'test':  ('test/images',  'test/labels'),
}

OLD_CLASS = '0'  # כל התוויות כיום כ־0 (patch)
NEW_CLASS = '7'  # index של Patch Repair בקובץ names המקורי (0–10)

for split, (img_sub, lbl_sub) in splits.items():
    lbl_dir = os.path.join(PATCH_BASE, lbl_sub)
    if not os.path.isdir(lbl_dir):
        print(f"⚠️ לא נמצאה תיקיית תוויות עבור {split}: {lbl_dir}")
        continue

    for lbl_path in glob.glob(os.path.join(lbl_dir, '*.txt')):
        # קורא את כל השורות, מחליף את ה-class index ושומר
        lines = open(lbl_path, 'r').read().splitlines()
        with open(lbl_path, 'w') as f:
            for line in lines:
                parts = line.split()
                if parts[0] == OLD_CLASS:
                    parts[0] = NEW_CLASS
                f.write(' '.join(parts) + '\n')

    print(f"✅ עדכנתי תוויות ב־{split}, תיקיית תוויות: {lbl_dir}")

print("✨ כל התוויות עודכנו ל-class_id=7")


✅ עדכנתי תוויות ב־train, תיקיית תוויות: /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/patchs-2/train/labels
✅ עדכנתי תוויות ב־valid, תיקיית תוויות: /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/patchs-2/valid/labels
✅ עדכנתי תוויות ב־test, תיקיית תוויות: /Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/patchs-2/test/labels
✨ כל התוויות עודכנו ל-class_id=7


In [ ]:
from ultralytics import YOLO
model = YOLO('road_damage_detection_last_version.pt')  # Load model
model.val(data = "/Users/shachafemanoel/Documents/Hazard-Detection/public/object_detecion_model/RDD2022-3/data.yaml")  # Validate model

## הורדת ההדאטה של המודל 
# RDD2020 Dataset

In [10]:

from roboflow import Roboflow
rf = Roboflow(api_key="BQvvxKrJPoAaNkv4ZXlr")
project = rf.workspace("new-workspace-07lef").project("patchs")
version = project.version(2)
dataset = version.download("yolov12")
                

loading Roboflow workspace...
loading Roboflow project...


Python(12890) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Extracting Dataset Version Zip to patchs-2 in yolov12:: 100%|██████████| 3198/3198 [00:01<00:00, 1807.96it/s]
